In [25]:
import pandas as pd

# Read in the cleaned and labelled data

In [26]:
cleaned_labelled_data = pd.read_csv('../../data/IMDB_datasets/clean_imdb_dataset.csv')

isolate review and label

In [27]:
just_review_and_label = cleaned_labelled_data.iloc[:, 0:2]

add column of reviews split into words

In [28]:
just_review_and_label['words'] = [0 for _ in range(just_review_and_label.shape[0])]


for review_index in range(just_review_and_label.shape[0]):
    just_review_and_label['words'][review_index] = just_review_and_label['review'][review_index].split()

C:\Users\luke0\AppData\Local\Temp\ipykernel_21168\1310866257.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  just_review_and_label['words'][review_index] = just_review_and_label['review'][review_index].split()
C:\Users\luke0\AppData\Loca

# Read in each of the three cleaned and NE tagged data

In [29]:
def read_conll_file(path):
    """
    read in conll file
    
    :param path: path to read from
    :returns: list with sequences of words and labels for each sentence
    """
    data = []
    current_words = []
    current_tags = []

    for line in open(path, encoding='utf-8'):
        line = line.strip()

        if line:
            if line[0] == '#':
                continue # skip comments
            tok = line.split('\t')

            current_words.append(tok[1])
            current_tags.append(tok[2])
        else:
            if current_words:  # skip empty lines
                data.append((current_words, current_tags))
            current_words = []
            current_tags = []

    # check for last one
    if current_tags != []:
        data.append((current_words, current_tags))
    return data

In [30]:
test_10 = read_conll_file('../../data/imdb_test_train_datasets/test_tagged/test_9010_tagged_output.iob2')

test_50 = read_conll_file('../../data/imdb_test_train_datasets/test_tagged/test_5050_tagged_output.iob2')

test_90 = read_conll_file('../../data/imdb_test_train_datasets/test_tagged/test_1090_tagged_output.iob2')

convert to dataframe

In [31]:
test_10_df = pd.DataFrame(test_10, columns = ['words', 'ne_tags'])
test_50_df = pd.DataFrame(test_50, columns = ['words', 'ne_tags'])
test_90_df = pd.DataFrame(test_90, columns = ['words', 'ne_tags'])

function that adds empty columns for sentiment, and converts the word list to a joined review

In [32]:
def prepare_space_for_sentiment(tagged_dataframe):

    size = tagged_dataframe.shape[0]

    tagged_dataframe['sentiment'] = [0 for _ in range(size)]
    tagged_dataframe['review'] = [0 for _ in range(size)]

    for point in range(size):
        tagged_dataframe['review'][point] = ' '.join(tagged_dataframe['words'][point])

    prepared_dataframe = tagged_dataframe

    return prepared_dataframe
    

In [33]:
test_10_prepped = prepare_space_for_sentiment(test_10_df)
test_50_prepped = prepare_space_for_sentiment(test_50_df)
test_90_prepped = prepare_space_for_sentiment(test_90_df)

C:\Users\luke0\AppData\Local\Temp\ipykernel_21168\4032396392.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  tagged_dataframe['review'][point] = ' '.join(tagged_dataframe['words'][point])
C:\Users\luke0\AppData\Local\Temp\ipykernel_21168

# Function that checks for equivalent reviews, and adds the labels to the tagged data so that we have 3 Cleaned, Tagged and Labelled test data sets

In [34]:
def check_for_twin_reviews(df_cleaned_and_tagged, df_cleaned_and_labelled = just_review_and_label):

    x = df_cleaned_and_tagged
    y = df_cleaned_and_labelled

    z = df_cleaned_and_tagged

    labelled_size = y.shape[0]
    tagged_size = x.shape[0]

    for tagged_index in range(tagged_size):
        tagged_words = x['words'][tagged_index]
        cropped_size = len(tagged_words)

        for labelled_index in range(labelled_size):
            labelled_words = y['words'][labelled_index]
    
            if tagged_words == labelled_words[0:cropped_size]:
                z['sentiment'][tagged_index] = y['sentiment'][labelled_index]

                break


    return z

In [35]:
test_50_tagged_and_labelled = check_for_twin_reviews(df_cleaned_and_tagged=test_50_prepped)

C:\Users\luke0\AppData\Local\Temp\ipykernel_21168\720714151.py:19: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  z['sentiment'][tagged_index] = y['sentiment'][labelled_index]
C:\Users\luke0\AppData\Local\Temp\ipykernel_21168\720714151.py:19:

In [36]:
def did_check_work(final_df):

    uh_oh = 0
    negs = 0
    possies = 0
    result = 'It flunked'

    for sentiment in final_df['sentiment']:
        if sentiment == 'negative':
            negs+=1
        elif sentiment == 'positive':
            possies +=1
        else:
            uh_oh+=1

    if uh_oh == 0:
        result = 'It worked'


    return (possies, negs, uh_oh, result)

In [37]:
did_check_work(test_50_tagged_and_labelled)

(12474, 12317, 0, 'It worked')

In [38]:
test_10_tagged_labelled = check_for_twin_reviews(df_cleaned_and_labelled=just_review_and_label, df_cleaned_and_tagged=test_10_prepped)

C:\Users\luke0\AppData\Local\Temp\ipykernel_21168\720714151.py:19: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  z['sentiment'][tagged_index] = y['sentiment'][labelled_index]
C:\Users\luke0\AppData\Local\Temp\ipykernel_21168\720714151.py:19:

In [39]:
did_check_work(final_df=test_10_tagged_labelled)

(2489, 2470, 0, 'It worked')

In [40]:
test_90_tagged_labelled = check_for_twin_reviews(df_cleaned_and_labelled=just_review_and_label, df_cleaned_and_tagged=test_90_prepped)

C:\Users\luke0\AppData\Local\Temp\ipykernel_21168\720714151.py:19: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  z['sentiment'][tagged_index] = y['sentiment'][labelled_index]
C:\Users\luke0\AppData\Local\Temp\ipykernel_21168\720714151.py:19:

In [41]:
did_check_work(final_df=test_90_tagged_labelled)

(22372, 22251, 0, 'It worked')

In [ ]:
# test_10_tagged_labelled.to_csv('../../data/imdb_test_train_datasets/test_tagged_and_labelled/test_10_tagged_and_labelled')
# test_50_tagged_and_labelled.to_csv('../../data/imdb_test_train_datasets/test_tagged_and_labelled/test_50_tagged_and_labelled')
# test_90_tagged_labelled.to_csv('../../data/imdb_test_train_datasets/test_tagged_and_labelled/test_90_tagged_and_labelled')